In [1]:
import pandas as pd

In [4]:
df_BCW = pd.read_csv('../../../datasets/BreastCancerWisconsinDataSet.csv')
df_BCW[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [5]:
df_BCW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [6]:
df_BCW.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

#### 목표변수
- 목표변수 : diagnosis
- 설명변수 : radius_worst, texture_worst, perimeter_worst

#### 전처리
- diagnosis 전처리(apply() M-0, B-1)

In [9]:
def change(value):
    if value == 'M':
        return 0
    else :
        return 1 

df_BCW['diagnosis'] = df_BCW['diagnosis'].apply(change)

In [11]:
df_BCW['diagnosis'].value_counts()

1    357
0    212
Name: diagnosis, dtype: int64

In [26]:
df_BCW_extract = df_BCW[['diagnosis', 'radius_worst', 'texture_worst', 'perimeter_worst']]
df_BCW_extract[10:15]

,diagnosis,radius_worst,texture_worst,perimeter_worst
10,0,19.19,33.88,123.8
11,0,20.42,27.28,136.5
12,0,20.96,29.94,151.7
13,0,16.84,27.66,112.0
14,0,15.03,32.01,108.8


In [13]:
df_BCW_extract.isnull().sum()

diagnosis          0
radius_worst       0
texture_worst      0
perimeter_worst    0
dtype: int64

In [15]:
# 정형화 단계
targets = df_BCW_extract['diagnosis']
features = df_BCW_extract[['radius_worst', 'texture_worst', 'perimeter_worst']]
targets.shape, features.shape

((569,), (569, 3))

In [18]:
# 모델 학습
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(features, targets)

LogisticRegression()

In [19]:
model.coef_, model.intercept_

(array([[ 0.27487967, -0.23812459, -0.23327034]]), array([27.3266644]))

In [33]:
# 미래예측
df_BCW_extract[50:55]

,diagnosis,radius_worst,texture_worst,perimeter_worst
50,1,12.98,25.72,82.98
51,1,14.67,23.19,96.08
52,1,13.10,21.33,83.67
53,0,20.60,24.13,135.10
54,0,18.10,31.69,117.70


In [34]:
# 11번째 값 14.67, 23.19, 96.08
temp_label = [[14.67, 23.19, 96.08]]

In [35]:
model.predict(temp_label)

C:\Users\01-02\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [20]:
# 평가
target_predict = model.predict(features)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
accuracy_score(targets,target_predict)

0.9490333919156415

In [24]:
# 서비스 배포
import pickle

In [25]:
with open('../../../datasets/BreastCancerWisconsin.pkl', 'wb') as regression_file :
    pickle.dump(obj=model, file=regression_file)
    pass

In [36]:
# Model 일반화 확인
# [[14.67, 23.19, 96.08]]
(0.27487967 * 14.67) + ( -0.23812459 * 23.19) + ( -0.23327034 * 96.08) + 27.3266644

3.4244256496